# **INTRODUCTIONS**

---

Used for exceptional processes.

The code is separated into multiple sections:

1. **[Views](#views)**
2. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**

---


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSSTAGE.MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS'))
CREATE TABLE [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS](
	[ID] [INT] IDENTITY (1,1) NOT NULL,
	[CATEGORY] [NVARCHAR](255) NULL,
	[KEY] [NVARCHAR](255) NULL,
	[VALUE] [NVARCHAR](255) NULL,
  	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE()
);
GO

-- 2) CREATE STAGE INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_STAGE_MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS')
CREATE NONCLUSTERED INDEX [IX_STAGE_MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS] ON [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
(
    [DATE_LAST_INSERTED] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

# **VIEWS**
---

### BOOSTER SHOT ADMINISTERED

Changes in sprint 42 for ticket **Automatiseren boosterprikdata \[BACKEND\]**

It is no longer necessary to manually insert these values so the view can be removed. 

Instead make use of <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">[VWSDEST].[RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL]</span>

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DROP VIEW IF EXISTS [VWSDEST].[V_MANUAL_BOOSTER_SHOT_ADMINISTERED]
GO


### THIRD SHOTS ADMINISTERED

In [ ]:

-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  VIEW [VWSDEST].[V_MANUAL_THIRD_SHOT_ADMINISTERED]
AS
	WITH CTE AS (
		SELECT *
		FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
		AND CATEGORY = 'THIRD_SHOT_ADMINISTERED'
	)
    SELECT
        -- Select columns,
		Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'ADMINISTERED_TOTAL')  AS ADMINISTERED_TOTAL,
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS DATE_UNIX,
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO

### BOOSTER AND THIRD SHOTS ADMINISTERED

Changes in sprint 42 for ticket **Automatiseren boosterprikdata \[BACKEND\]**

It is no longer necessary to manually insert these values so the view can be removed. 

Instead make use of <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">[VWSDEST].[RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL]</span>

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DROP VIEW IF EXISTS  [VWSDEST].[V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED]
GO


### BOOSTER SHOTS PLANNED

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE  OR ALTER    VIEW [VWSDEST].[V_MANUAL_BOOSTER_SHOT_PLANNED]
AS
	WITH CTE AS (
		SELECT *
		FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
		AND CATEGORY = 'BOOSTER_SHOT_PLANNED'
	)
    SELECT
        -- SELECT COLUMNS,
        Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'PLANNED_7_DAYS') AS PLANNED_7_DAYS,
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS [DATE_UNIX],
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO

### BOOSTER COVERAGE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE VIEW(S).....

--AS OF 18-03-2022 BELOW QUERY IS INVALID
/* CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL]
AS
    WITH CTE AS (
        SELECT
            *
        FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
            AND CATEGORY = 'BOOSTER_COVERAGE'
            
    )
    SELECT TOP(1)
        (SELECT ROUND(CAST([VALUE] AS FLOAT), 1) FROM CTE WHERE [KEY] = 'RECEIVED_BOOSTER_PERCENTAGE') AS [PERCENTAGE],
        dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS [DATE_UNIX],
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO */

# **DROP ARTIFACTS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT BOOSTER DATA',
        @proto_name VARCHAR(50) = 'NL';

-- 1) DROP PROTOS CONFIGURATION(S).....
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [VIEW_ID] IN (
    SELECT [ID] AS VID FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_Handmatig_BOOSTER_SHOT_PLANNED', 
        'VWSDEST.V_Handmatig_BOOSTER_AND_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_Handmatig_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_Handmatig_BOOSTER_SHOT_ADMINISTERED',
        'VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED', 
        'VWSDEST.V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_MANUAL_BOOSTER_SHOT_ADMINISTERED',
        'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
        'VWSDEST.V_RIVM_VACCINATIONS_THIRD_SHOTS_ADMINISTERED_NL',
        'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_AND_THIRD_SHOTS_ADMINISTERED_NL',
        --'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
        'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_PLANNED_NL'
    ))
    AND [PROTO_ID] IN (
        SELECT [ID] FROM [DATATINO_PROTO_1].[PROTOS] protos 
        WHERE [NAME] = @proto_name
    ) AND [NAME] IN (
        'booster_shot_planned',
        'booster_and_third_shot_administered',
        'third_shot_administered',
        'booster_shot_administered',
        'booster_coverage',
        'booster_shot_planned_DEACTIVATED',
        'booster_and_third_shot_administered_DEACTIVATED',
        'third_shot_administered_DEACTIVATED',
        'booster_shot_administered_DEACTIVATED',
        'booster_coverage_DEACTIVATED'
    );

DELETE FROM [DATATINO_PROTO_1].[VIEWS]
WHERE [ID] IN (
    SELECT 
        [ID] AS VID 
    FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_Handmatig_BOOSTER_SHOT_PLANNED', 
        'VWSDEST.V_Handmatig_BOOSTER_AND_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_Handmatig_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_Handmatig_BOOSTER_SHOT_ADMINISTERED',
        'VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED', 
        'VWSDEST.V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED', 
        'VWSDEST.V_MANUAL_BOOSTER_SHOT_ADMINISTERED',
        'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
        'VWSDEST.V_RIVM_VACCINATIONS_THIRD_SHOTS_ADMINISTERED_NL',
        'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_AND_THIRD_SHOTS_ADMINISTERED_NL',
        --'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
        'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_PLANNED_NL'
    )
)

-- 2) DROP PROCESS(ES)......
;WITH CTE 
AS
(
    SELECT
        v_processes.[PROCESS_NAME],
        processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] IN (
        'PROCESS_LOAD_MANUAL_INSERT_BOOSTER_DATA'
    ) AND v_workflows.[NAME] = @workflow_name
)
DELETE [DATATINO_ORCHESTRATOR_1].[PROCESSES] 
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
WHERE dataflows.[NAME] IN (SELECT [PROCESS_NAME] FROM CTE) 
    AND processes.[ID] IN (SELECT [ID] FROM CTE);

;WITH CTE 
AS
(
    SELECT
        v_processes.[PROCESS_NAME],
        processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] IN (
        'PROCESS_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
        'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_STAGE_TO_INTER',
        'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_INTER_TO_DEST'
    ) AND v_workflows.[NAME] = 'RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL'
)
DELETE [DATATINO_ORCHESTRATOR_1].[PROCESSES] 
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
WHERE dataflows.[NAME] IN (SELECT [PROCESS_NAME] FROM CTE) 
    AND processes.[ID] IN (SELECT [ID] FROM CTE);


-- 4) DROP SOURCE(S).....
DELETE FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] IN (
    'SOURCE_LOAD_MANUAL_INSERT_BOOSTER_DATA'
);

-- 5) DROP WORKFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[WORKFLOWS]
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
WHERE dataflows.[NAME] = @workflow_name
    AND workflows.[ID] IN (
        SELECT
            workflows.[ID]
        FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
        INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] 
            AND v_workflows.[ID] = workflows.[ID]
        WHERE v_workflows.[NAME] = @workflow_name
    );

-- 8) DEPENDENCIES
;WITH CTE 
AS 
(
    SELECT 
            [ID] FROM [DATATINO_ORCHESTRATOR_1].V_DEPENDENCIES
    WHERE [MAIN_NAME] IN (
        'PROCESS_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
        'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_STAGE_TO_INTER',
        'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_INTER_TO_DEST'
    ) OR [DEPENDENCY_NAME] IN (
        'PROCESS_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
        'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_STAGE_TO_INTER',
        'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_INTER_TO_DEST'
    ) AND [WORKFLOW_NAME] = 'RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL'
)
DELETE [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] 
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
WHERE dependencies.[ID] IN (SELECT [ID] FROM CTE);

-- 6) DROP DATAFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[H_DATAFLOWS]
WHERE [NAME] IN (
    'PROCESS_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    'PROCESS_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
    'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_STAGE_TO_INTER',
    'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_INTER_TO_DEST',
    @workflow_name
);

DELETE  [DATATINO_ORCHESTRATOR_1].[DATAFLOWS]
WHERE [NAME] IN (
    'PROCESS_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    'PROCESS_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL',
    'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_STAGE_TO_INTER',
    'PROCESS_SP_RIVM_VACCINATIONS_BOOSTER_SHOTS_ADMINISTERED_NL_INTER_TO_DEST',
    @workflow_name
)
GO

# **DATATINO CONFIGURATIONS**
---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT BOOSTER DATA'

-- 1) UPSERT WORKFLOW(S).....
EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = null, 
    @workflow_name = @workflow_name,
    @description = 'MANUAL INSERT OF BOOSTER DATA', 
    @schedule = '0 13 * * *', -- DAILY AT 01:00 PM
    @active = 1;

-- 2) UPSERT SOURCE(S).....
EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE] 
    @id = null,
    @source_name = 'SOURCE_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    @description = 'SOURCE MANUAL INSERT OF BOOSTER DATA',
    @source = 'datafiles/VWS/Handmatige_invoer_Booster_data.csv',
    @source_columns = 'CATEGORY|KEY|VALUE',
    @target_columns = 'CATEGORY|KEY|VALUE',
    @target_name = 'VWSSTAGE.MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS',
    @source_type = 'CsvFile',
    @location_type = 'AzureBlob',
    @delimiter_type = 'SemiColon',
    @security_profile= 'RIVM';

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_LOAD_MANUAL_INSERT_BOOSTER_DATA';
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = 'SOURCE_LOAD_MANUAL_INSERT_BOOSTER_DATA';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;  
GO

### PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT BOOSTER DATA';

--VIEW THIRD_SHOT_ADMINISTERED
-- 1) UPSERT PROTO VIEW(S).....
    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @description = 'VIEW OF V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null;

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'VIEW CONFIGURATION FOR VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @view_name = 'VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @item_name = 'third_shot_administered',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1';


--VIEW V_MANUAL_BOOSTER_SHOT_PLANNED
-- 1) UPSERT PROTO VIEW(S).....
    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED',
    @description = 'VIEW OF V_MANUAL_BOOSTER_SHOT_PLANNED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null;

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'VIEW CONFIGURATION FOR VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED',
    @view_name = 'VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED',
    @item_name = 'booster_shot_planned',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1';

--VIEW V_MANUAL_BOOSTER_COVERAGE
/* -- 1) UPSERT PROTO VIEW(S).....
    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @description = 'VIEW OF V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null;

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'VIEW CONFIGURATION FOR VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @view_name = 'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @item_name = 'booster_coverage',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1';
GO */